### Universal Decomposition (Partial)
This file completes the decomposition of a general unitary matrix into two-level unitary matrices. 

The full Universal Decomposition requires another fixed routine to decompose two-level unitary matrices into CNOT and single-qubit gates. Since this procedure is not so interesting and takes only $O(1)$ time, we leave it our for simplicity. 

In [1]:
import numpy as np

def reduce(a, b): 
    '''
    Reduce elements (i, i+1) by U
    '''
    if b == 0:
        return np.identity(2, dtype=complex)
    if a == 0: 
        return np.array([[0,1], [1,0]],  dtype=complex)
    t = np.arctan2(np.abs(b), np.abs(a)) # theta
    l = -np.angle(a)          # lambda
    m = np.pi + np.angle(b)   # mu  
    U = np.array( [[ np.cos(t)*np.exp(l*1j), np.sin(t)*np.exp(m*1j)],
                   [-np.sin(t)*np.exp(-m*1j), np.cos(t)*np.exp(-l*1j)]])
    return U 

def gray_code(n):
    return [i^i//2 for i in range(2**n)]

def two_level_decompose(A):
    '''
    Decomposes A into a list of 2-level matrices such that the product 
    of the list will be A. 
    '''
    assert A.shape[0] == A.shape[1]
    d = A.shape[0]
    decomp = []
    for s in range(d-2):
        for i in range(d,s+1,-1):
            b = A[s][i-1]
            a = A[s][i-2]
            I = np.identity(d, dtype=np.cdouble)
            U = reduce(a, b)
            for j in range(2):
                for k in range(2): 
                    I[i-2+j][i-2+k] = U[j][k]    
            decomp.append(I.conj().T)
            A = A @ I
    decomp.append(A)
    return decomp

def two_level_decompose_gray(A): 
    '''
    Decomposes A into a sequence of 2-level matrices such that acts on
    basis states that diff. in only one bit.
    '''
    assert A.shape[0] == A.shape[1]
    d = A.shape[0]
    P = np.zeros((d, d))
    gcode = gray_code(int(np.log2(d)))
    
    for i in range(d): 
        for j in range(d): 
            if gcode[j] == i:
                P[i][j] = 1
    D = two_level_decompose(P @ A @ P.T)
    for i in range(len(D)):
        D[i] = P.T @ D[i] @ P
    return D 

        
def YZ_decompose(U): 
    '''
    Single qubit gate U can be implemented using four gates. 
    U = R_1(phi) R_z(l+u) R_y(2t) R_z(l-u)
    
    R_1(a) = [[1,0],[0,e^ia]]
    R_y = exp(iaY/2)
    R_z = exp(iaZ/2)
    '''
    assert U.shape == (2,2)
    phi = np.angle( np.linalg.det(U) ) 
    t = np.arccos(np.abs(U[0][0])) 
    l = np.angle(U[0][0])
    u = np.angle(U[0][1])




## Test Universal Decomposition

In [2]:
from scipy.stats import unitary_group

n = 3
targetGate = unitary_group.rvs(2**n)
np.set_printoptions(suppress=True, precision=3)

# Test to recover a target gate to be decomposed
D = two_level_decompose_gray(targetGate)
S = np.identity(targetGate.shape[0])
for m in D:
    S = m @ S

# this should be a zero matrix 
print(S-targetGate)

[[-0.+0.j -0.+0.j  0.-0.j -0.-0.j  0.-0.j -0.+0.j  0.+0.j  0.+0.j]
 [-0.+0.j  0.-0.j  0.-0.j  0.+0.j -0.-0.j  0.+0.j  0.-0.j -0.+0.j]
 [ 0.-0.j  0.-0.j -0.+0.j -0.+0.j  0.-0.j -0.+0.j -0.-0.j  0.+0.j]
 [-0.+0.j  0.+0.j  0.-0.j  0.+0.j -0.-0.j  0.+0.j  0.+0.j -0.-0.j]
 [ 0.-0.j -0.-0.j  0.-0.j  0.+0.j  0.-0.j  0.-0.j  0.+0.j  0.+0.j]
 [-0.+0.j -0.+0.j  0.-0.j -0.-0.j  0.+0.j  0.+0.j -0.+0.j  0.-0.j]
 [-0.+0.j -0.-0.j  0.+0.j  0.-0.j  0.+0.j  0.-0.j  0.+0.j  0.+0.j]
 [ 0.-0.j -0.-0.j -0.+0.j  0.+0.j  0.-0.j  0.+0.j -0.-0.j  0.+0.j]]


### Computer Experiment & Complexity Analysis

In [3]:
from time import perf_counter
import csv

print( len( two_level_decompose(targetGate) ) )
iterations  = 1
max_nqubits = 7

with open('analysisUD.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['nqubits', 'avg l', 'avg runtime'])
    for n in range(1, max_nqubits+1): 
        total_l = 0
        total_t = 0 
        for i in range(iterations):
            targetGate = unitary_group.rvs(2**n)
            stime = perf_counter() 
            D = two_level_decompose_gray(targetGate)
            duration = perf_counter() - stime
            l = len(D)
            total_l += l 
            total_t += duration
            # print("length", l)
            # print("duration", duration)
        writer.writerow( [n, total_l/iterations ,total_t/iterations])

28
